In [ ]:
import os, sys
CWD = os.getcwd()
if os.path.basename(CWD) == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(CWD, ".."))
else:
    PROJECT_ROOT = CWD  # fallback if already at root
os.chdir(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "utils"))
print(f"Correct Working Directory: {str(os.path.basename(os.getcwd()))=='battery_xct_notebooks'}")


import tifffile as tiff
import numpy as np
import skimage
from skimage.measure import label, find_contours, EllipseModel, ransac
from skimage.draw import polygon2mask
from scipy.ndimage import center_of_mass as c_of_m
from scipy.ndimage import distance_transform_edt as dist_trans
from scipy.ndimage import convolve
import matplotlib.pyplot as plt
import math
import utils.plotting_utils as plot
import utils.can as cn

px = 0.03397 # mm

In [ ]:
# Read image and calcualte a mask
im = tiff.imread('data/cell_vol/im075.tif')
mask = im > 100

# Side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 10))
axes = axes.ravel()

# Iamge Mask Pair: image + overlay
axes[0].imshow(im, cmap="gray", interpolation="nearest")
axes[0].set_title("XCT Slice")
axes[0].axis("off")
plot.show_overlay(im, mask, axes[1], "XCT Slice + Can Mask")

plt.tight_layout()
plt.show()

In [ ]:
def extract_inner_outer_contours(mask, as_mask = False):
    contours = find_contours((mask).astype(float), 0.5)
    outer = max(contours, key=lambda c: 0.5*np.abs(np.sum(c[:,1]*np.roll(c[:,0],-1) - c[:,0]*np.roll(c[:,1],-1))))
    inner = min(contours, key=lambda c: 0.5*np.abs(np.sum(c[:,1]*np.roll(c[:,0],-1) - c[:,0]*np.roll(c[:,1],-1))))
    if as_mask == True:
        outer = polygon2mask(mask.shape, outer).astype(np.uint8)
        inner = polygon2mask(mask.shape, inner).astype(np.uint8)
    return outer, inner

outer, inner = extract_inner_outer_contours(mask, as_mask = True)
inner = inner * ~mask # Remove any 'wall' pixels from inner mask

fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(outer+inner, interpolation="none")
axes.set_title("Inner + Outer Mask")
axes.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
def equivalent_diameter_area(mask):
    area = np.sum(mask)
    return math.sqrt(area/math.pi)*2

OD = equivalent_diameter_area(outer)
print(f'Outer Diameter: {(OD*px):.2f} mm')

ID = equivalent_diameter_area(inner)
print(f'Inner Diameter: {(ID*px):.2f} mm')

In [ ]:
def surface_pixels_convolve(img):
    img8 = img.astype(np.uint8)
    s = convolve(img8, np.ones((3,3), np.uint8), mode='constant', cval=0)
    # interior: all 9 in the 3x3 (8-connected)
    interior = (img & (s == 9))
    return img & ~interior

outer_border = surface_pixels_convolve(outer)
outer_border = (outer_border == 1).astype(bool)

inner_wall = surface_pixels_convolve(inner)
inner_wall = (inner_wall == 1).astype(bool)

edt = dist_trans(np.where(outer_border == True, False, True))
thickness_vals = edt[inner_wall]
thick_mean, thick_min, thick_max, thick_std = thickness_vals.mean(), thickness_vals.min(), thickness_vals.max(), thickness_vals.std()

print(f'Mean Wall Thickness: {(thick_mean*px):.3f} mm')
print(f'Wall Thickness 3\u03C3: {(thick_std*3*px):.3f} mm')

In [ ]:
contours = find_contours((mask).astype(float), 0.5)
outer = max(contours, key=lambda c: 0.5*np.abs(np.sum(c[:,1]*np.roll(c[:,0],-1) - c[:,0]*np.roll(c[:,1],-1))))
pts_xy = np.column_stack([outer[:,1], outer[:,0]])  # (x,y)

model, inliers = ransac(
    pts_xy,
    EllipseModel,
    min_samples=5,
    residual_threshold=1.0,     # tunable
    max_trials=2000,
    # stop_probability=0.999,   # optional
)
xc, yc, a, b, phi = model.params

In [ ]:
plot.plot_ellipse_on_mask(im, model)

In [ ]:
A, B = (a, b) if a >= b else (b, a)   # ensure A = major, B = minor
eccentricity = np.sqrt(max(0.0, 1.0 - (B/A)**2))  # guard tiny numerical negatives

print(f"Eccentricity : {eccentricity:.4f}")

In [ ]:
# radii & angles of observed contour w.r.t. the ellipse center
vx = pts_xy[:, 0] - xc
vy = pts_xy[:, 1] - yc
theta = np.arctan2(vy, vx)                     # [-pi, pi)
r_obs = np.hypot(vx, vy)

In [ ]:
# rotate the direction into the ellipse's principal frame
ct = np.cos(theta - phi)
st = np.sin(theta - phi)
r_elli = (a * b) / np.sqrt((b * ct) ** 2 + (a * st) ** 2)


In [ ]:
# dent depth (positive = inward dent; negative = outward bulge)
delta = r_elli - r_obs

In [ ]:
# bin by angle for a uniform profile
n_bins = 180
bins = np.linspace(-np.pi, np.pi, n_bins + 1)
centers = (bins[:-1] + bins[1:]) / 2
idx = np.digitize(theta, bins) - 1
idx[idx == n_bins] = 0  # wrap edge case

prof = np.full(n_bins, np.nan)
for k in range(n_bins):
    m = (idx == k)
    if np.any(m):
        prof[k] = np.nanmean(delta[m])

dmax = float(np.nanmax(prof)*px)
print(f'max denting : {dmax:.4f} mm')

### Synthetic Data Example 

In [ ]:
# Read image and calcualte a mask
mask = tiff.imread('data/ellipse_dent.tif')
px = 0.0479

# Side-by-side subplots
fig, axes = plt.subplots(1, 1, figsize=(6, 6))

# Iamge Mask Pair: image + overlay
axes.imshow(mask, cmap="gray", interpolation="nearest")
axes.set_title("Ellipse with Dent (Synthetic data)")
axes.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
help(cn.calculate_can_metrics)

In [ ]:
# Example
metrics, model = cn.calculate_can_metrics(
    mask,
    use_ransac=True,
    ransac_residual_threshold=1.0,
    ransac_max_trials=3000,
    n_bins=180, 
    profile_agg="median", 
    pixel_size=px,
    return_model=True,
)

print(f"OD = {metrics['diameter']['outer_equiv_units']:.2f} mm")
print(f"ID = {metrics['diameter']['inner_equiv_units']:.2f} mm")
print(f"Thickness mean/min/max = "
      f"{metrics['thickness']['mean_units']:.2f} / "
      f"{metrics['thickness']['min_units']:.2f} / "
      f"{metrics['thickness']['max_units']:.2f} mm")
print(f"Eccentricity = {metrics['ellipse']['eccentricity']:.4f}")
print(f"Max dent = {metrics['dent']['max_units']:.2f} mm")

plot.plot_ellipse_on_mask(mask, model)
